# Enunciado

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets

## Preparación de datos (1.0pt)

Explore el archivo `covid19_confirmados.csv` y utilice la función `pd.read_csv` para importarlo como un `DataFrame` considerando las siguientes indicaciones:

1. Utilice el nombre de país como índice.
1. Elimine las columnas `Lat` y `Long`.
1. Use TimeStamps para las columnas asociadas a fechas.
1. Reduzca la columna `Province/State`. El DataFrame resultante debe tener los totales a nivel país.

In [ ]:
data = pd.read_csv('data/covid19_confirmados.csv',
                   index_col='Country/Region'
                 )
data.drop(['Lat','Long'],axis=1,inplace=True)

data_confirmados = data.groupby('Country/Region',sort=True).sum()
data_confirmados.columns = pd.to_datetime(data_confirmados.columns)

Explore el archivo `population_by_country_2020.csv` y utilice la función `pd.read_csv` de pandas para importarlo como un `DataFrame`. Utilice el nombre de país como índice principal. 

Nota: Algunos siglas o nombres podrían no calzar con el `DataFrame` anterior, explore y corrija programaticamente dichos errores.

In [ ]:
d_popu = pd.read_csv('data/poblacion_mundial2020.csv',
                   index_col='Country (or dependency)'
                 )
data_popu = d_popu.rename_axis('Country/Region')

In [ ]:
filtro = data_confirmados.merge(data_popu, how='outer',on='Country/Region', indicator='Filtro')
paises_1 = filtro[filtro.Filtro == 'left_only'].index.to_list()

# Visualizar los países del primer dataframe que no coinciden
paises_1

In [ ]:
paises_2 = filtro[filtro.Filtro == 'right_only'].index.to_list()

# Visualizar los países del segundo dataframe que no coinciden
paises_2

In [ ]:
# Arreglo de los nombres, haciendo coincidir los países con error de escritura
data_popu.rename({'United States' : 'US',
                  'DR Congo' : 'Congo (Kinshasa)',
                  'Myanmar' : 'Burma',
                  'South Korea' : 'Korea, South',
                  "Côte d'Ivoire" : "Cote d'Ivoire",
                  'Taiwan' : 'Taiwan*',
                  'Czech Republic (Czechia)' : 'Czechia',
                  'Congo' : 'Congo (Brazzaville)',
                  'Sao Tome & Principe' : 'Sao Tome and Principe',
                  'St. Vincent & Grenadines' : 'Saint Vincent and the Grenadines',
                  'Saint Kitts & Nevis' : 'Saint Kitts and Nevis', 
                 }, axis=0, inplace=True)

Realice un `merge` de los `DataFrame` anteriores. El objetivo es asignar un valor de `Population` a cada elemento del primer `DataFrame`. Indique las filas donde no se puede realizar el `merge` y luego descártelas de su `DataFrame` final.

In [ ]:
data_conf_popu_filter = data_confirmados.merge(data_popu, how='outer', left_index=True, right_index=True, indicator='filtro')
data_conf_popu_filter.drop(['Yearly Change', 'Net Change',
       'Density (P/Km²)', 'Land Area (Km²)', 'Migrants (net)',
       'Fert. Rate', 'Med. Age', 'Urban Pop %', 'World Share'],
        axis=1,
        inplace=True)

# Encontrar países donde no hay coincidencias
paises_no_coincidentes = data_conf_popu_filter[data_conf_popu_filter.filtro == 'right_only'].index.to_list() +  data_conf_popu_filter[data_conf_popu_filter.filtro == 'left_only'].index.to_list()
print('Los países no coincidentes son: \n', paises_no_coincidentes)

In [ ]:
data_conf_popu = data_confirmados.merge(data_popu, how='outer', left_index=True, right_index=True)
data_conf_popu.drop(['Yearly Change', 'Net Change',
       'Density (P/Km²)', 'Land Area (Km²)', 'Migrants (net)',
       'Fert. Rate', 'Med. Age', 'Urban Pop %', 'World Share'],
        axis=1,
        inplace=True)

# Eliminar del dataframe las filas donde no coinciden las columnas de interés
data_conf_popu.dropna()

Repita el procedimiento para los archivos `covid19_recuperados.csv` y `covid19_muertes.csv`.

In [ ]:
# Importar datas
data_muerte = pd.read_csv('data/covid19_muertes.csv', index_col='Country/Region')
data_muerte.drop(['Lat','Long'],axis=1,inplace=True)
data_muertes = data_muerte.groupby('Country/Region',sort=False).sum()
data_muertes.columns = pd.to_datetime(data_muertes.columns)


data_recu = pd.read_csv('data/covid19_recuperados.csv', index_col='Country/Region')
data_recu.drop(['Lat','Long'],axis=1,inplace=True)
data_recuperados = data_recu.groupby('Country/Region',sort=False).sum()
data_recuperados.columns = pd.to_datetime(data_recuperados.columns)

# Merge muertes-poblacion
data_mrt_popu = data_muertes.merge(data_popu, how='inner', left_index=True, right_index=True)
data_mrt_popu.drop(['Yearly Change', 'Net Change',
       'Density (P/Km²)', 'Land Area (Km²)', 'Migrants (net)',
       'Fert. Rate', 'Med. Age', 'Urban Pop %', 'World Share'],
        axis=1,
        inplace=True)

# Merge recuperados-poblacion
data_recu_popu = data_recuperados.merge(data_popu, how='inner', left_index=True, right_index=True)
data_recu_popu.drop(['Yearly Change', 'Net Change',
       'Density (P/Km²)', 'Land Area (Km²)', 'Migrants (net)',
       'Fert. Rate', 'Med. Age', 'Urban Pop %', 'World Share'],
        axis=1,
        inplace=True)

data_mrt_popu.dropna()
data_recu_popu.dropna();

## Análisis global (1.5pt)

Escriba una función que reciba una fecha y que retorne las siguientes series: 

1. Tasa de incidencia: Casos confirmados por 100.000 dividido población total.
1. Tasa de recuperación: Casos recuperados por 100.000 dividido población total.
1. Tasa de mortalidad: Casos decesos por 100.000 dividido población total.

Ahora, considerando los siguientes dos puntos temporales:

- 22 de Enero de 2021.
- 22 de Enero de 2022.

Muestre los nombres y los valores de los 3 países con:

1. La mayor y menor tasa de incidencia, respectivamente.
1. La mayor y menor tasa de recuperación, respectivamente.
1. La mayor y menor tasa de mortalidad, respectivamente.

Muestre también la posición de Chile en el ranking.

In [ ]:
# Tasa de incidencia confirmados:
def Tasas(f):
    c = (data_conf_popu[f]*100000)/data_conf_popu['Population (2020)']
    r = (data_recu_popu[f]*100000)/data_conf_popu['Population (2020)']
    m = (data_mrt_popu[f]*100000)/data_conf_popu['Population (2020)']
    
    return c,r,m

In [ ]:
# Análisis del 1/22/21:
t_incidencia_1_22_21,t_recuperados_1_22_21,t_muerte_1_22_21 = Tasas(pd.to_datetime('01/22/2021'))
t_incidencia_1_22_21_sorted = t_incidencia_1_22_21.sort_values(ascending=False)
t_recuperados_1_22_21_sorted = t_recuperados_1_22_21.sort_values(ascending=False)
t_muerte_1_22_21_sorted = t_muerte_1_22_21.sort_values(ascending=False)

print('La mayor tasa de incidencia es de: ',t_incidencia_1_22_21.idxmax(),'con ', t_incidencia_1_22_21.max())
print('La menor tasa de incidencia es de: ',t_incidencia_1_22_21.idxmin(),'con ', t_incidencia_1_22_21.min())

print('La mayor tasa de recuperados es de: ',t_recuperados_1_22_21.idxmax(),'con ', t_recuperados_1_22_21.max())
print('La menor tasa de recuperados es de: ',t_recuperados_1_22_21.idxmin(),'con ', t_recuperados_1_22_21.min())

print('La mayor tasa de muerte es de: ',t_muerte_1_22_21.idxmax(),'con ', t_muerte_1_22_21.max())
print('La menor tasa de muerte es de: ',t_muerte_1_22_21.idxmin(),'con ', t_muerte_1_22_21.min())

In [ ]:
# Encontrar posicion de Chile para enero de 2021
for p in range(t_incidencia_1_22_21_sorted.size):
    if t_incidencia_1_22_21_sorted.index[p] == 'Chile':
        print('Chile se encuentra en la posición',p+1,'de las tasas de incidencia más altas para enero de 2021')
    if t_recuperados_1_22_21_sorted.index[p] == 'Chile':
        print('Chile se encuentra en la posición',p+1,'de las tasas de recuperados más altas para enero de 2021')
    if t_muerte_1_22_21_sorted.index[p] == 'Chile':
        print('Chile se encuentra en la posición',p+1,'de las tasas de muerte más altas para enero de 2021')

In [ ]:
# Análisis del 1/22/22:
t_incidencia_1_22_22,t_recuperados_1_22_22,t_muerte_1_22_22 = Tasas(pd.to_datetime('01/22/2022'))
t_incidencia_1_22_22_sorted = t_incidencia_1_22_22.sort_values(ascending=False)
t_recuperados_1_22_22_sorted = t_recuperados_1_22_22.sort_values(ascending=False)
t_muerte_1_22_22_sorted = t_muerte_1_22_22.sort_values(ascending=False)

print('La mayor tasa de incidencia es de: ',t_incidencia_1_22_22.idxmax(),'con ', t_incidencia_1_22_22.max())
print('La menor tasa de incidencia es de: ',t_incidencia_1_22_22.idxmin(),'con ', t_incidencia_1_22_22.min())

print('La mayor tasa de recuperados es de: ',t_recuperados_1_22_22.idxmax(),'con ', t_recuperados_1_22_22.max())
print('La menor tasa de recuperados es de: ',t_recuperados_1_22_22.idxmin(),'con ', t_recuperados_1_22_22.min())

print('La mayor tasa de muerte es de: ',t_muerte_1_22_22.idxmax(),'con ', t_muerte_1_22_22.max())
print('La menor tasa de muerte es de: ',t_muerte_1_22_22.idxmin(),'con ', t_muerte_1_22_22.min())

In [ ]:
# Encontrar posicion de Chile para enero de 2022
for p in range(t_incidencia_1_22_22_sorted.size):
    if t_incidencia_1_22_22_sorted.index[p] == 'Chile':
        print('Chile se encuentra en la posición',p+1,'de las tasas de incidencia más altas para enero de 2022')
    if t_recuperados_1_22_22_sorted.index[p] == 'Chile':
        print('Chile se encuentra en la posición',p+1,'de las tasas de recuperados más altas para enero de 2022')
    if t_muerte_1_22_22_sorted.index[p] == 'Chile':
        print('Chile se encuentra en la posición',p+1,'de las tasas de muerte más altas para enero de 2022')

Ahora realice las siguientes gráficas de nube de punto

- Tasa de incidencia Enero 2021 versus Tasa de incidencia Enero 2022.
- Tasa de mortalidad Enero 2021 versus Tasa de incidencia Enero 2022.

En cada una:

- Utilice color para demarcar el [continente](https://www.kaggle.com/datasets/statchaitya/country-to-continent) de cada país.
- Realice una anotación con el código del pais cercano a su punto: https://country-code.cl/es/.
- Utilice un tamaño relativamente más grande para Chile.
- Considere las buenas prácticas vistas en clases.

In [ ]:
continentes = pd.read_csv('countryContinent.csv', usecols=['country','continent','code_2'], index_col='country')
tablas = continentes.rename_axis('Country/Region')
tablas['incidencia 2021'] = t_incidencia_1_22_21
tablas['incidencia 2022'] = t_incidencia_1_22_22
tablas['mortalidad 2021'] = t_muerte_1_22_21
tablas.dropna();

In [ ]:
table_reset_index = tablas.reset_index( level= 0,drop=False,inplace=False)
table_reset_index.dropna(inplace=True);
table_reset_index.set_index(['continent','Country/Region'],inplace=True)
table_reset_index;

In [ ]:
# Tasa de incidencia Enero 2021 versus Tasa de incidencia Enero 2022.
%matplotlib inline

idx = pd.IndexSlice

fig, ax = plt.subplots(figsize=(10, 6), tight_layout=True)

ax.scatter(x=table_reset_index.loc[idx['Asia',:],'incidencia 2021'],
           y=table_reset_index.loc[idx['Asia',:],'incidencia 2022'],s=5,c='r',alpha=0.5,label='Continente Asiático')

ax.scatter(x=table_reset_index.loc[idx['Europe',:],'incidencia 2021'],
           y=table_reset_index.loc[idx['Europe',:],'incidencia 2022'],s=5,c='b',alpha=0.5,label='Continente Europeo')

ax.scatter(x=table_reset_index.loc[idx['Americas',:],'incidencia 2021'],
           y=table_reset_index.loc[idx['Americas',:],'incidencia 2022'],s=5,c='magenta',alpha=0.5,label='Continente Americano')

ax.scatter(x=table_reset_index.loc[idx['Oceania',:],'incidencia 2021'],
           y=table_reset_index.loc[idx['Oceania',:],'incidencia 2022'],s=5,c='orange',alpha=0.5,label='Continente Oceánico')

ax.scatter(x=table_reset_index.loc[idx['Africa',:],'incidencia 2021'],
           y=table_reset_index.loc[idx['Africa',:],'incidencia 2022'],s=5,c='g',alpha=0.5,label='Continente Africano')

ax.scatter(x=table_reset_index.loc[idx[:,'Chile'],'incidencia 2021'],
           y=table_reset_index.loc[idx[:,'Chile'],'incidencia 2022'],s=100,c='k',alpha=0.5,label='Chile')

c=0
for x in table_reset_index.iloc[:,0]:
    label= x
    plt.annotate(label,(table_reset_index.iloc[c,1],table_reset_index.iloc[c,2]),ha='center',fontsize=6)
    c=c+1

ax.set_xlabel('Incidencia enero 2021',fontsize=13)
ax.set_ylabel('Incidencia enero 2022',fontsize=13)
ax.legend();

In [ ]:
# Tasa de mortalidad Enero 2021 versus Tasa de incidencia Enero 2022.
%matplotlib inline

idx = pd.IndexSlice

fig, ax = plt.subplots(figsize=(10, 6), tight_layout=True)

ax.scatter(x=table_reset_index.loc[idx['Asia',:],'mortalidad 2021'],
           y=table_reset_index.loc[idx['Asia',:],'incidencia 2022'],s=5,c='r',alpha=0.5,label='Continente Asiático')

ax.scatter(x=table_reset_index.loc[idx['Europe',:],'mortalidad 2021'],
           y=table_reset_index.loc[idx['Europe',:],'incidencia 2022'],s=5,c='b',alpha=0.5,label='Continente Europeo')

ax.scatter(x=table_reset_index.loc[idx['Americas',:],'mortalidad 2021'],
           y=table_reset_index.loc[idx['Americas',:],'incidencia 2022'],s=5,c='magenta',alpha=0.5,label='Continente Americano')

ax.scatter(x=table_reset_index.loc[idx['Oceania',:],'mortalidad 2021'],
           y=table_reset_index.loc[idx['Oceania',:],'incidencia 2022'],s=5,c='orange',alpha=0.5,label='Continente Oceánico')

ax.scatter(x=table_reset_index.loc[idx['Africa',:],'mortalidad 2021'],
           y=table_reset_index.loc[idx['Africa',:],'incidencia 2022'],s=5,c='g',alpha=0.5,label='Continente Africano')

ax.scatter(x=table_reset_index.loc[idx[:,'Chile'],'mortalidad 2021'],
           y=table_reset_index.loc[idx[:,'Chile'],'incidencia 2022'],s=100,c='k',alpha=0.5,label='Chile')

c=0
for x in table_reset_index.iloc[:,0]:
    label= x
    plt.annotate(label,(table_reset_index.iloc[c,3],table_reset_index.iloc[c,2]),fontsize=5)
    c=c+1

ax.set_xlabel('Mortalidad enero 2021',fontsize=13)
ax.set_ylabel('Incidencia enero 2022',fontsize=13)
ax.legend();

En base a los rankings y gráficas de nube de punto analice y discuta sobre el caso de Chile considerando las siguientes preguntas guía:

- ¿A qué países del mundo se parece más?
- ¿A qué países de América Latina se parece más?
- ¿A qué países de la [OCDE](https://es.wikipedia.org/wiki/Organizaci%C3%B3n_para_la_Cooperaci%C3%B3n_y_el_Desarrollo_Econ%C3%B3micos) se parece más? 


## Análisis comparativo (1.5pt)

Construya una interfaz gráfica usando `ipywidgets` y `matplotlib` que permita realizar comparaciones entre las series de tiempo de COVID-19 de dos países seleccionados arbitrariamente. A modo de simplificación considere sólo los países de la [OCDE](https://es.wikipedia.org/wiki/Organizaci%C3%B3n_para_la_Cooperaci%C3%B3n_y_el_Desarrollo_Econ%C3%B3micos). 

Requerimientos:

1. La interfaz debe incluir controles para escoger dos países a partir de una lista.
1. La interfaz debe incluir tres [tabs](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Tabs), el primero debe mostrar las series de tiempo de ambos países de los casos confirmados, el segundo la de recuperados y el tercero la de los decesos, respectivamente. Use una leyenda para indicar el nombre de los países.
1. La interfaz debe tener un control que permita escoger entre casos acumulados y casos nuevos. Hint: Para los casos nuevos pueden revisar la función [`diff`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.diff.html).
1. La interfaz debe tener un control que permita escoger entre valores absolutos y valores relativos (tasas por 100.000 habitantes).
1. La interfaz debe tener un control que permita escoger entre valores diarios y valores semanales. Para esto último se recomienda usar `groupby` con una [frecuencia lunes a lunes](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#anchored-offsets).
    



In [ ]:
#Lista con paises de la OCDE
%matplotlib ipympl
paises =["Austria", "Belgium", "Canada", "Denmark", "France", "Germany", "Greece", "Ireland", "Iceland", "Italy", "Luxembourg", "Netherlands", "Norway", "Portugal", "Spain", "Sweden", "Switzerland", "Turkey", "United Kingdom", "US", "Japan", "Finland", "Australia", "New Zealand", "Mexico", "Czech Republic (Czechia)", "North Korea", "Hungary", "Poland", "Slovak Republic", "Chile", "Israel", "Slovenia", "Estonia", "Latvia", "Lithuania", "Colombia", "Costa Rica"]
paises_dicc = {pais: pais for pais in paises}  #diccionario con paises de la OCDE para mas comodidad
poblacion = data_popu["Population (2020)"]

wid = widgets.Dropdown(description = "Pais 1:", options=paises, value = None)
wid2 = widgets.Dropdown(description = "Pais 2: ", options=paises, value = None )
wid3 = widgets.Dropdown(description = "Casos: ", options = ["Acumulados", "Nuevos"], value = None)
wid4 = widgets.Dropdown(description = "Valores: ", options = ["Absolutos", "Relativos"],value = None)
wid5 = widgets.Dropdown(description = "Tiempo: ", options = ["Diarios", "Semanales"],value = None)

button = widgets.Button(
    description='Actualizar grafico',
    disabled=False,
    tooltip='Click me',
)


tab_contents = ['Casos Confirmados', 'Casos Recuperados', 'Casos Decesos']
variable_tabs = widgets.Tab()
variable_tabs.children = [widgets.Output() for i in range(len(tab_contents))]
for i in range(len(tab_contents)):
    variable_tabs.set_title(i, tab_contents[i])

variable_tabs.selected_index = None
def update_graph(button):
        if (wid.value != None or wid3.value == "Acumulados" or wid4.value == "Absolutos"):
            pais_1_confirmados = data_confirmados.loc[paises_dicc[wid.value]]
            pais_1_recuperados = data_recuperados.loc[paises_dicc[wid.value]]
            pais_1_decesos = data_muertes.loc[paises_dicc[wid.value]]

        if (wid2.value != None or wid3.value == "Acumulados" or wid4.value == "Absolutos"):
            pais_2_confirmados = data_confirmados.loc[paises_dicc[wid2.value]]
            pais_2_recuperados = data_recuperados.loc[paises_dicc[wid2.value]]
            pais_2_decesos = data_muertes.loc[paises_dicc[wid2.value]]

        if (wid3.value == "Nuevos"):
            pais_1_confirmados = data_confirmados.loc[paises_dicc[wid.value]].diff().fillna(0)
            pais_1_recuperados = data_recuperados.loc[paises_dicc[wid.value]].diff().fillna(0)
            pais_1_decesos = data_muertes.loc[paises_dicc[wid.value]].diff().fillna(0)

            pais_2_confirmados = data_confirmados.loc[paises_dicc[wid2.value]].diff().fillna(0)
            pais_2_recuperados = data_recuperados.loc[paises_dicc[wid2.value]].diff().fillna(0)
            pais_2_decesos = data_muertes.loc[paises_dicc[wid2.value]].diff().fillna(0)
        
        if (wid4.value == "Relativos"):
            poblacion_1 = poblacion.loc[paises_dicc[wid.value]]
            poblacion_2 = poblacion.loc[paises_dicc[wid2.value]]
            pais_1_confirmados = data_confirmados.loc[paises_dicc[wid.value]] / poblacion_1 * 100000
            pais_1_recuperados = data_confirmados.loc[paises_dicc[wid.value]] / poblacion_1 * 100000
            pais_1_decesos = data_confirmados.loc[paises_dicc[wid.value]] / poblacion_1 * 100000

            pais_2_confirmados = data_confirmados.loc[paises_dicc[wid2.value]] / poblacion_2 * 100000
            pais_2_recuperados = data_confirmados.loc[paises_dicc[wid2.value]] / poblacion_2 * 100000
            pais_2_decesos = data_confirmados.loc[paises_dicc[wid2.value]] / poblacion_2 * 100000

        if (wid5.value == "Diarios"):
            pais_1_confirmados = pais_1_confirmados.diff().fillna(0)
            pais_1_recuperados = pais_1_recuperados.diff().fillna(0)
            pais_1_decesos = pais_1_decesos.diff().fillna(0)

            pais_2_confirmados = pais_2_confirmados.diff().fillna(0)
            pais_2_recuperados = pais_2_recuperados.diff().fillna(0)
            pais_2_decesos = pais_2_decesos.diff().fillna(0)

        elif (wid5.value == "Semanales"):
            pais_1_confirmados = pais_1_confirmados.groupby(pd.Grouper(freq='W-MON')).sum()
            pais_1_recuperados = pais_1_recuperados.groupby(pd.Grouper(freq='W-MON')).sum()
            pais_1_decesos = pais_1_decesos.groupby(pd.Grouper(freq='W-MON')).sum()

            pais_2_confirmados = pais_2_confirmados.groupby(pd.Grouper(freq='W-MON')).sum()
            pais_2_recuperados = pais_2_recuperados.groupby(pd.Grouper(freq='W-MON')).sum()
            pais_2_decesos = pais_2_decesos.groupby(pd.Grouper(freq='W-MON')).sum()
        plt.figure(figsize=(10, 6))
        
        if (variable_tabs.selected_index == 0):
            plt.plot(pais_1_confirmados.index, pais_1_confirmados.values, label=wid.value)
            plt.plot(pais_2_confirmados.index, pais_2_confirmados.values, label=wid2.value)
            plt.title(tab_contents[variable_tabs.selected_index])
            plt.xlabel('Fecha')
            plt.ylabel('Confirmados')
            plt.legend()
            plt.show()
            

        if (variable_tabs.selected_index == 1):
            plt.plot(pais_1_recuperados.index, pais_1_recuperados.values, label=wid.value)
            plt.plot(pais_2_recuperados.index, pais_2_recuperados.values, label=wid2.value)
            plt.title(tab_contents[variable_tabs.selected_index])
            plt.xlabel('Fecha')
            plt.ylabel('Recuperados')
            plt.legend()
            plt.show()
        
        if (variable_tabs.selected_index == 2):
            plt.plot(pais_1_decesos.index, pais_1_decesos.values, label=wid.value)
            plt.plot(pais_2_decesos.index, pais_2_decesos.values, label=wid2.value)
            plt.title(tab_contents[variable_tabs.selected_index])
            plt.xlabel('Fecha')
            plt.ylabel('Muertes')
            plt.legend()
            plt.show()


button.on_click(update_graph)
display(wid, wid2, wid3, wid4, wid5, variable_tabs, button)
update_graph(None)
plt.clf()

## Dashboard interactivo (1.0pt)

Implemente un *dashboard web* interactivo a partir de la interfaz desarrollada en el punto anterior utilizando la librería [voila](https://github.com/voila-dashboards/voila). Sirva su interfaz en la nube, para esto puede considerar los servicios gratuitos [binder](https://mybinder.org/) o [heroku](https://www.heroku.com/free). Revise la documentación de voila [con respecto a *deployment* en la nube](https://voila.readthedocs.io/en/stable/deploy.html) y este repositorio con [un ejemplo en base a heroku](https://github.com/voila-dashboards/voila-heroku). 

Escriba el link de su dashboard web a continuación:

## Discusión final (1.0pt)

En base a su interfaz:   

1. Describa en detalle la situación de Chile con respecto a la evolución temporal de las series de tiempo. Destaque patrones y comportamientos que considere relevantes. Compare lo que observa en las distintas series de tiempo. 
1. Considerando la evolución a nivel de semana, conteste ¿En qué fechas ocurre un aumento o disminución notoria de los casos? ¿En qué fechas hay máximos y mínimos?
1. Considerando métricas que sean relativas compare el caso de Chile contra cinco países de la OCDE seleccionados por ustedes. Destaque semejanzas y diferencias considerando las cantidades y los posibles desfases temporales existentes.